# Data Wrangling Project: Wrangle and Analys Data

## Intorduction

In this project I'm  going to apply the data wrangling techniques that I've learned on some data that I will collect from Twitter using tweepy API, and after I'm done wrangling the data I will analyse it and create a visualization to help understand how data wrangling can affect the final outcome for any analysis.

I will follow the same steps that I learned in the data wrangling lesson to reach my goal, the steps are:

* Gathering the data
* Assessing the data
* Cleaning the data

after I'm done with these steps I will start analysing the cleaned data and compare it to the unclean data analysis.

=====================================================================================================================

## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy as tw
import json

## Gathering

For the gathering we will accuire three types of data:

1. CSV data from the "Twitter-Archive-Enhanced"
2. TSV data generated from the Image prediction algorithim
3. JSON data from the Tweepy API

In [2]:
# Start with the csv file using read_csv function.
df_csv = pd.read_csv('twitter-archive-enhanced.csv')
df_csv.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
# Read the tsv file also using read_csv function but changing the sep variable
df_tsv = pd.read_csv("image-predictions.tsv",sep='\t')
df_tsv.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [4]:
# Authenticating and using the Tweepy API

consumer_key = '1ifPTGNs26JtJqirQFlyd4Jz1'
consumer_secret = 'ZR9382UxmSWgmwXX1PocpsyWHniotQYotMw1N8kJc0Pz4aG449'
access_token = '786162021561532417-waB9mIK8Ov8YcCQZJyAzg7yKz5QsFD5'
access_token_secret = 'SY145NwAH9m78MGhmIQW9gLh7hN7S4aUHNkwpMaEoJQhs'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

# Storing the Tweet IDs to use them with the API
tweet_ids = df_csv['tweet_id'].values

In [5]:
# Checking the user for the twitter account that we're intrested in
user = api.get_user('dog_rates')
print(user.name)
print(user.description)

WeRateDogs™
Your Only Source For Professional Dog Ratings Instagram and Facebook ➪ WeRateDogs partnerships@weratedogs.com ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀


In [6]:
with open('tweet_json.txt', 'w') as file:
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, file)
            file.write('\n')
        except tw.TweepError:
            print('Failed to return json object for tweet ID: {}'.format(tweet_id))

Failed to return json object for tweet ID: 888202515573088257
Failed to return json object for tweet ID: 873697596434513921
Failed to return json object for tweet ID: 872668790621863937
Failed to return json object for tweet ID: 872261713294495745
Failed to return json object for tweet ID: 869988702071779329
Failed to return json object for tweet ID: 866816280283807744
Failed to return json object for tweet ID: 861769973181624320
Failed to return json object for tweet ID: 856602993587888130
Failed to return json object for tweet ID: 851953902622658560
Failed to return json object for tweet ID: 845459076796616705
Failed to return json object for tweet ID: 844704788403113984
Failed to return json object for tweet ID: 842892208864923648
Failed to return json object for tweet ID: 837366284874571778
Failed to return json object for tweet ID: 837012587749474308
Failed to return json object for tweet ID: 829374341691346946
Failed to return json object for tweet ID: 827228250799742977
Failed t

In [6]:
# Extracting JSON objects from the text file

tweets_list =[]


with open('tweet_json.txt') as json_file:
    for line in json_file:
        tweets_dict = {}
        tweets_json = json.loads(line)
        # Using a json viewer we can see that tweet_id is in extended_entities => media => id
        try:
            tweets_dict['tweet_id'] = tweets_json['extended_entities']['media'][0]['id']
        except:
            tweets_dict['tweet_id'] = 'na'

        tweets_dict['retweet_count'] = tweets_json['retweet_count']
        tweets_dict['favorite_count'] = tweets_json['favorite_count']
        
        tweets_list.append(tweets_dict)

In [7]:
df_json = pd.DataFrame(tweets_list)
df_json.head()

,favorite_count,retweet_count,tweet_id
0,37362,8037,892420639486877696
1,32115,5966,892177413194625024
2,24214,3948,891815175371796480
3,40665,8205,891689552724799489
4,38882,8885,891327551943041024


==========================================================================

## Assessing

For the assessing we will look for these issues in each data frame:

1. Check for missing data
2. Check for problems with the data structure
3. Check for quality issues

### Assessing the <span style="color:green">CSV</span> file. 

In [8]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
df_csv.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [10]:
df_csv.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [11]:
df_csv.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [12]:
df_csv.groupby(['rating_denominator']).count()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,name,doggo,floofer,pupper,puppo
rating_denominator,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1
2,1,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1
7,1,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1
10,2333,73,73,2333,2333,2333,180,180,180,2278,2333,2333,2333,2333,2333,2333
11,3,0,0,3,3,3,1,1,1,3,3,3,3,3,3,3
15,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1
16,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1
20,2,1,1,2,2,2,0,0,0,1,2,2,2,2,2,2
40,1,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1


In [13]:
df_csv.query('rating_denominator != 10')['rating_denominator'].count()

23

In [14]:
df_csv.query('rating_denominator != 10')['rating_numerator']

313     960
342      11
433      84
516      24
784       9
902     165
1068      9
1120    204
1165      4
1202     50
1228     99
1254     80
1274     45
1351     60
1433     44
1598      4
1634    143
1635    121
1662      7
1663     20
1779    144
1843     88
2335      1
Name: rating_numerator, dtype: int64

In [15]:
df_csv.groupby(['rating_numerator']).count()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_denominator,name,doggo,floofer,pupper,puppo
rating_numerator,,,,,,,,,,,,,,,,
0,2,1,1,2,2,2,0,0,0,2,2,2,2,2,2,2
1,9,3,3,9,9,9,1,1,1,6,9,9,9,9,9,9
2,9,0,0,9,9,9,0,0,0,9,9,9,9,9,9,9
3,19,0,0,19,19,19,0,0,0,19,19,19,19,19,19,19
4,17,1,1,17,17,17,0,0,0,16,17,17,17,17,17,17
5,37,2,2,37,37,37,1,1,1,35,37,37,37,37,37,37
6,32,0,0,32,32,32,0,0,0,32,32,32,32,32,32,32
7,55,2,2,55,55,55,1,1,1,53,55,55,55,55,55,55
8,102,0,0,102,102,102,4,4,4,102,102,102,102,102,102,102


In [16]:
check = df_csv.query('rating_numerator < 10 | rating_numerator > 14')[['text','rating_numerator']]
check.to_csv("checkingNumerator.csv",index=True)

In [17]:
df_csv['tweet_id'].duplicated().sum()

0

In [18]:
df_csv['doggo'].unique()

array(['None', 'doggo'], dtype=object)

In [19]:
df_csv.query('doggo == "None"& floofer == "None" & pupper == "None" & puppo == "None"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
10,890006608113172480,NaN,NaN,2017-07-26 00:31:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Koda. He is a South Australian decksha...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890006608...,13,10,Koda,None,None,None,None


In [20]:
df_csv.query('(doggo != "None"& floofer != "None") | (pupper != "None" & puppo != "None")')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
200,854010172552949760,NaN,NaN,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...","At first I thought this was a shy doggo, but i...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/854010172...,11,10,None,doggo,floofer,None,None


In [21]:
df_csv.query('(doggo != "None"& pupper != "None") | (floofer != "None" & puppo != "None")')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686...,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460...,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,https://twitter.com/dog_rates/status/733109485...,12,10,None,doggo,None,pupper,None
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,https://twitter.com/dog_rates/status/741067306...,12,10,just,doggo,None,pupper,None
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/759793422...,12,10,Maggie,doggo,None,pupper,None
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending it pictures that don't eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751583847...,5,10,None,doggo,None,pupper,None


#### Result of Assessment for the CSV file:

* Visually looking at retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,in_reply_to_status_id,in_reply_to_user_id we can see that these columns have lots of missing values.
* We found that there are 23 entries a rating_denominator that is not 10,  which is a clear error(we might not need this column since we know our ratings are out of 10).
* Some of the rating_numerator are either less than 10 or greater than 14, after visually checking these entries I found that some of these values are due to error in reading the text(this could occur for text containing dates (e.g. 4/10(4th July) or places (e.g. 7/11).
* column names are ambiguous
* timestamp and retweeted_status_timestamp are not in the correct data type
* Last 4 columns (doggo, floofer,pupper and puppo) should be in one column instead of four.
---

### Assessing the <span style="color:red">TSV</span> file.

In [22]:
df_tsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [23]:
df_tsv.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [24]:
df_tsv.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [25]:
df_tsv.isnull().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [26]:
df_tsv['tweet_id'].duplicated().sum()

0

In [27]:
df_tsv['jpg_url'].duplicated().sum()

66

In [28]:
df_tsv['img_num'].unique()

array([1, 4, 2, 3])

In [29]:
1-df_tsv['p1_dog'].mean()

0.2616867469879518

In [30]:
df_tsv.query('p1_conf == {}'.format(df_tsv['p1_conf'].min()))

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
38,666644823164719104,https://pbs.twimg.com/media/CUBl6IwVAAA9_zT.jpg,1,Ibizan_hound,0.044333,True,Pembroke,0.043209,True,West_Highland_white_terrier,0.038906,True


#### Result of Assessment for the TSV file:

* Visually looking at the data, I see multiple columns that I don't need in my final cleaned data (e.g. p2 data and p3 data).
* 26% of the predictions in p1 are false, which is supposed to be the strongest prediction.
* column names must have meaningful names
----

### Assessing the <span style="color:orange">JSON</span> file.

In [31]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 3 columns):
favorite_count    2334 non-null int64
retweet_count     2334 non-null int64
tweet_id          2334 non-null object
dtypes: int64(2), object(1)
memory usage: 54.8+ KB


In [32]:
df_json.head()

,favorite_count,retweet_count,tweet_id
0,37362,8037,892420639486877696
1,32115,5966,892177413194625024
2,24214,3948,891815175371796480
3,40665,8205,891689552724799489
4,38882,8885,891327551943041024


In [33]:
df_json.describe()

,favorite_count,retweet_count
count,2334.000000,2334.000000
mean,7787.058698,2825.568123
std,12083.495333,4781.927851
min,0.000000,1.000000
25%,1353.750000,568.250000
50%,3392.500000,1322.500000
75%,9558.250000,3297.500000
max,161352.000000,81174.000000


In [34]:
df_json[df_json['tweet_id'].duplicated()]

,favorite_count,retweet_count,tweet_id
31,0,104,na
34,19751,3542,na
41,70375,19405,na
54,123,7,na
63,302,10,na
67,0,6480,na
71,29203,6812,na
72,0,1230,na
73,0,6353,na
82,22802,4443,na


In [35]:
df_json.query('tweet_id == 675501041127936000')

,favorite_count,retweet_count,tweet_id
582,0,5920,675501041127936000
1836,17291,5920,675501041127936000


#### Result of Assessment for the JSON file:

* There are entries without tweet_id
* There are duplicated tweet_id
* tweet_id is in object data type

----------------------------------------------
==========================================================================

## Cleaning

Cleaning consists of three main steps:
1. Defining
2. Coding
3. Testing

### Defining 

We will define how we will solve the issues we found in the assessment phase.

#### Result of Assessment for the CSV file:

* Visually looking at retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,in_reply_to_status_id,in_reply_to_user_id we can see that these columns have lots of missing values.  
<span style="color:turquoise">Remove all of these columns since we will not be using them.</span>
* We found that there are 23 entries a rating_denominator that is not 10,  which is a clear error(we might not need this column since we know our ratings are out of 10).  
<span style="color:turquoise">Remove this column since we know that the denominator should always be 10.</span>
* Some of the rating_numerator are either less than 10 or greater than 14, after visually checking these entries I found that some of these values are due to error in reading the text(this could occur for text containing dates (e.g. 4/10(4th July) or places (e.g. 7/11).    
<span style="color:turquoise">Fix entries that are due to problems in readign the text, then remove them if they are actual errors</span>
* column names are ambiguous  
<span style="color:turquoise">Change the columns name to meaningful ones.</span>
* timestamp and retweeted_status_timestamp are not in the correct data type  
<span style="color:turquoise">Change data type to datetime for these columns.</span>
* Last 4 columns (doggo, floofer,pupper and puppo) should be in one column instead of four.  
<span style="color:turquoise">First create a column called "stage" that holds the value for each entry stage(one or multiple).</span>

#### Result of Assessment for the TSV file:

* Visually looking at the data, I see multiple columns that I don't need in my final cleaned data (e.g. p2 data and p3 data).  
<span style="color:turquoise">Remove all of these columns since we will not be using them.</span>
* 26% of the predictions in p1 are false, which is supposed to be the strongest prediction.  
<span style="color:turquoise">Remove these entries since they should be always True.</span>
* column names must have meaningful names  
<span style="color:turquoise">Change them to meaningful names.</span>

#### Result of Assessment for the JSON file:

* There are entries without tweet_id  
<span style="color:turquoise">Drop these entries</span>
* There are duplicated tweet_id  
<span style="color:turquoise">Remove the ones that have no favorite_count or retweet_count.</span>
* tweet_id is in object data type  
<span style="color:turquoise">Change data type to int.</span>

### Coding & Testing

Now we will tackle all of the issues that we found during the assessment, but before we begin we must make a copy of the three data frames

In [36]:
df_csv_clean = df_csv.copy()
df_tsv_clean = df_tsv.copy()
df_json_clean = df_json.copy()

#### Cleaning the <span style="color:green">CSV</span> file. 

In [37]:
# drop unnecessary columns
df_csv_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','in_reply_to_status_id','in_reply_to_user_id'], axis=1,inplace=True)
df_csv_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [38]:
# Remove rating_denominator since it should be always 10, and we will not use it anyways
df_csv_clean.drop(['rating_denominator'],axis=1,inplace=True)
df_csv_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,Franklin,None,None,None,None


In [39]:
# there are 11 entries that are wrong due to errors in reading the text
df_csv_clean.at[45, 'rating_numerator'] = 13
df_csv_clean.at[55, 'rating_numerator'] = 17
df_csv_clean.at[313, 'rating_numerator'] = 13
df_csv_clean.at[763, 'rating_numerator'] = 11
df_csv_clean.at[784, 'rating_numerator'] = 14
df_csv_clean.at[1068, 'rating_numerator'] = 14
df_csv_clean.at[1165, 'rating_numerator'] = 13
df_csv_clean.at[1202, 'rating_numerator'] = 11
df_csv_clean.at[1459, 'rating_numerator'] = 13
df_csv_clean.at[1662, 'rating_numerator'] = 10
df_csv_clean.at[1712, 'rating_numerator'] = 11
df_csv_clean.loc[[45,55,313,1165,1202],'rating_numerator']

45      13
55      17
313     13
1165    13
1202    11
Name: rating_numerator, dtype: int64

In [40]:
# remove rows with rating_numerator greater than 14 or less than 10
df_csv_clean.drop(df_csv_clean.query('rating_numerator < 10 | rating_numerator > 14').index,axis=0,inplace=True)
df_csv_clean.query('rating_numerator < 10 | rating_numerator > 14')

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,name,doggo,floofer,pupper,puppo


In [41]:
# give each column a better name
df_csv_clean.rename(columns={'timestamp': 'tweet_time', 'text': 'tweet_text','rating_numerator': 'dog_rating/10', 'name': 'dog_name'},inplace=True)
df_csv_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1898 entries, 0 to 2350
Data columns (total 11 columns):
tweet_id         1898 non-null int64
tweet_time       1898 non-null object
source           1898 non-null object
tweet_text       1898 non-null object
expanded_urls    1855 non-null object
dog_rating/10    1898 non-null int64
dog_name         1898 non-null object
doggo            1898 non-null object
floofer          1898 non-null object
pupper           1898 non-null object
puppo            1898 non-null object
dtypes: int64(2), object(9)
memory usage: 177.9+ KB


In [42]:
# change tweet_time from object to datetime
df_csv_clean['tweet_time']= pd.to_datetime(df_csv_clean['tweet_time']) 
df_csv_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1898 entries, 0 to 2350
Data columns (total 11 columns):
tweet_id         1898 non-null int64
tweet_time       1898 non-null datetime64[ns]
source           1898 non-null object
tweet_text       1898 non-null object
expanded_urls    1855 non-null object
dog_rating/10    1898 non-null int64
dog_name         1898 non-null object
doggo            1898 non-null object
floofer          1898 non-null object
pupper           1898 non-null object
puppo            1898 non-null object
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 177.9+ KB


In [43]:
# Replace None with null for last 4 columns and create a stage column that contains the dog stage

df_csv_clean['doggo'] = df_csv_clean['doggo'].replace('None','')
df_csv_clean['floofer'] = df_csv_clean['floofer'].str.replace('None','')
df_csv_clean['pupper'] = df_csv_clean['pupper'].str.replace('None','')
df_csv_clean['puppo'] = df_csv_clean['puppo'].str.replace('None','')
df_csv_clean['stage'] = df_csv_clean['doggo']+df_csv_clean['floofer']+df_csv_clean['pupper']+df_csv_clean['puppo']
df_csv_clean['stage'].value_counts()

                1566
pupper           203
doggo             79
puppo             28
doggopupper       11
floofer            9
doggopuppo         1
doggofloofer       1
Name: stage, dtype: int64

In [44]:
# put a comma between stages with multiple values
df_csv_clean.loc[df_csv_clean['stage'] == 'doggopupper', 'stage'] = 'doggo, pupper'
df_csv_clean.loc[df_csv_clean['stage'] == 'doggofloofer', 'stage'] = 'doggo, floofer'
df_csv_clean.loc[df_csv_clean['stage'] == 'doggopuppo', 'stage'] = 'doggo, puppo'


In [45]:
# remove the 4 stage columns
df_csv_clean.drop(['doggo','floofer','pupper','puppo'],axis=1,inplace=True)

In [46]:
df_csv_clean.sample(6)

,tweet_id,tweet_time,source,tweet_text,expanded_urls,dog_rating/10,dog_name,stage
677,789280767834746880,2016-10-21 01:42:53,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Meyer. He has to hold s...,https://twitter.com/dog_rates/status/750719632...,12,Meyer,
1437,697242256848379904,2016-02-10 02:14:42,"<a href=""http://twitter.com/download/iphone"" r...",This is Oakley. He has a massive tumor growing...,https://twitter.com/dog_rates/status/697242256...,10,Oakley,
672,789903600034189313,2016-10-22 18:57:48,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Ralphy. His dreams were just shattered...,https://vine.co/v/5wPT1aBxPQZ,13,Ralphy,pupper
1205,715928423106027520,2016-04-01 15:46:52,"<a href=""http://twitter.com/download/iphone"" r...",This is Bubbles. He's a Yorkshire Piccolope. 1...,https://twitter.com/dog_rates/status/715928423...,11,Bubbles,
1936,673956914389192708,2015-12-07 20:07:04,"<a href=""http://twitter.com/download/iphone"" r...",This is one esteemed pupper. Just graduated co...,https://twitter.com/dog_rates/status/673956914...,10,one,pupper
714,784057939640352768,2016-10-06 15:49:14,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Balto. He's very content. Legendary to...,https://vine.co/v/5gKxeUpuKEr,12,Balto,


In [47]:
df_csv_clean['tweet_id'].duplicated().sum()

0

#### Cleaning the <span style="color:red">TSV</span> file. 

In [48]:
# removing unnecessary columns
df_tsv_clean.drop(['jpg_url', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], inplace=True, axis=1)
df_tsv_clean.head()

,tweet_id,img_num,p1,p1_conf,p1_dog
0,666020888022790149,1,Welsh_springer_spaniel,0.465074,True
1,666029285002620928,1,redbone,0.506826,True
2,666033412701032449,1,German_shepherd,0.596461,True
3,666044226329800704,1,Rhodesian_ridgeback,0.408143,True
4,666049248165822465,1,miniature_pinscher,0.560311,True


In [49]:
# remove rows with a false p1_dog
df_tsv_clean.drop(df_tsv_clean.query('p1_dog == False').index,axis=0,inplace=True)
df_tsv_clean.query('p1_dog == False')

,tweet_id,img_num,p1,p1_conf,p1_dog


In [50]:
#  remove the column p1_dog, it has served it's purpose
df_tsv_clean.drop(['p1_dog'],axis=1,inplace=True)
df_tsv_clean.head()

,tweet_id,img_num,p1,p1_conf
0,666020888022790149,1,Welsh_springer_spaniel,0.465074
1,666029285002620928,1,redbone,0.506826
2,666033412701032449,1,German_shepherd,0.596461
3,666044226329800704,1,Rhodesian_ridgeback,0.408143
4,666049248165822465,1,miniature_pinscher,0.560311


In [51]:
# change the names of the columns
df_tsv_clean.rename(columns={'p1':'predicted_dog_breed', 'p1_conf':'prediction_confidence'} , inplace=True)
df_tsv_clean.head()

,tweet_id,img_num,predicted_dog_breed,prediction_confidence
0,666020888022790149,1,Welsh_springer_spaniel,0.465074
1,666029285002620928,1,redbone,0.506826
2,666033412701032449,1,German_shepherd,0.596461
3,666044226329800704,1,Rhodesian_ridgeback,0.408143
4,666049248165822465,1,miniature_pinscher,0.560311


In [52]:
df_tsv_clean['tweet_id'].duplicated().sum()

0

#### Cleaning the <span style="color:orange">JSON</span> file. 

In [53]:
# remove rows with no tweet_id
df_json_clean.drop(df_json_clean.query('tweet_id == "na"').index,axis=0,inplace=True)
df_json_clean.query('tweet_id == "na"')

,favorite_count,retweet_count,tweet_id


In [54]:
df_json_clean['tweet_id'].count()

2062

In [55]:
# remove duplicated 60 tweet_id where favorite_count is 0

ids = df_json_clean['tweet_id']
dupid = df_json_clean[ids.isin(ids[ids.duplicated()])].sort_values("tweet_id")
df_json_clean.drop(dupid.query('favorite_count == 0').index,axis=0,inplace=True)
df_json_clean['tweet_id'].count()

2002

In [56]:
df_json_clean['tweet_id'].duplicated().sum()

0

In [57]:
# change data type of tweet_id to int64
df_json_clean['tweet_id'] = df_json_clean['tweet_id'].astype(np.int64)
df_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2002 entries, 0 to 2333
Data columns (total 3 columns):
favorite_count    2002 non-null int64
retweet_count     2002 non-null int64
tweet_id          2002 non-null int64
dtypes: int64(3)
memory usage: 62.6 KB


In [58]:
df_json_clean['retweet_count'].describe()

count     2002.000000
mean      2543.450549
std       4569.277727
min         11.000000
25%        562.250000
50%       1222.500000
75%       2880.500000
max      81174.000000
Name: retweet_count, dtype: float64

## Merging the data frames

In [59]:
# Merging dataframes to create a master dataframe 

df_combined = pd.merge(df_csv_clean, df_tsv_clean, on='tweet_id', how='left')
df_combined2 = pd.merge(df_combined, df_json_clean, on='tweet_id',how='left')
df_combined2.head()

,tweet_id,tweet_time,source,tweet_text,expanded_urls,dog_rating/10,dog_name,stage,img_num,predicted_dog_breed,prediction_confidence,favorite_count,retweet_count
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,Phineas,,NaN,NaN,NaN,NaN,NaN
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,Tilly,,1.0,Chihuahua,0.323581,NaN,NaN
2,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,Archie,,1.0,Chihuahua,0.716012,NaN,NaN
3,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,Darla,,NaN,NaN,NaN,NaN,NaN
4,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,Franklin,,2.0,basset,0.555712,NaN,NaN


In [60]:
df_combined2['retweet_count'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: retweet_count, dtype: float64

In [61]:
# backup the combined data frame
df_combined2.to_csv('twitter_archive_master.csv')
df_csv_clean.to_csv('cleaned_twitter_archive_data.csv')
df_tsv_clean.to_csv('cleaned_image_prediction_data.csv')
df_json_clean.to_csv('cleaned_retweets_favoraites_data.csv')